In [15]:
import gc
import pandas as pd
import scipy.sparse as sps
import numpy as np

In [16]:
URM_test=sps.load_npz("./dressipi_dataset/URM_test_leaderboard.npz")

In [17]:
URM_test.shape

(4439758, 23692)

In [18]:
np.ravel(URM_test.sum(axis=1)).shape

(4439758,)

In [19]:
Session_ids=np.argwhere(np.ravel(URM_test.sum(axis=1)))

In [20]:
is_sorted = lambda a: np.all(a[:-1] <= a[1:])

In [21]:
is_sorted(Session_ids)

True

In [22]:
dataset=None
dataset_max=None
#["UCF","ICBF","ICF","Graph","TopPop100","TopPop50","TopPop15"]
for file in ["UCF","ICBF","Graph","ICF", "TopPop15"]:
    print(gc.collect())
    print(f"loading {file}")
    df=pd.read_csv(f"./leaderboard_ranker_dataset/{file}.csv")
    print(len(df))
    print(f"loaded {file}")
    for col in df:
        if "Max" in col:
            df_max=df[["Session_Id",col]].drop_duplicates()
        elif "Score" in col:
            df_feats=df[["Session_Id","Item_ID",col]]
    if dataset is None:
        dataset=df_feats
        dataset_max=df_max
    else:
        dataset=dataset.merge(df_feats,on=["Session_Id","Item_ID"],how="outer")
        dataset_max=dataset_max.merge(df_max,on=["Session_Id"],how="outer")
    del df


del df_feats
del df_max

2293
loading UCF
15952528
loaded UCF
0
loading ICBF
14870776
loaded ICBF
0
loading Graph
8315774
loaded Graph
0
loading ICF
17772188
loaded ICF
0
loading TopPop15
7500000
loaded TopPop15


In [23]:
ids=np.sort(dataset["Session_Id"].unique())
map_ids=pd.DataFrame()
map_ids["Session_Id"]=ids
map_ids["Real_Id"]=Session_ids

In [24]:
dataset_max=dataset_max.merge(map_ids,on="Session_Id",how="left")
del dataset_max["Session_Id"]
dataset_max.rename(columns={"Real_Id":"Session_Id"},inplace=True)

In [25]:
dataset=dataset.merge(map_ids,on="Session_Id",how="left")
del dataset["Session_Id"]
dataset.rename(columns={"Real_Id":"Session_Id"},inplace=True)

In [26]:
import ast
submission_df=pd.read_csv("./candidate-nn/leaderboard_candidates.csv")
submission_df["score"]=submission_df["score"].map(ast.literal_eval)
submission_df["code"]=submission_df["code"].map(ast.literal_eval)
submission_df["rank"]=submission_df["rank"].map(ast.literal_eval)
submission_df = submission_df.explode(['code','score', 'rank'])
submission_df[["session_id","code","rank"]]=submission_df[["session_id","code","rank"]].astype(int)
submission_df.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_gru","score":"score_gru"},inplace=True)

In [27]:
import ast
submission_df_tr=pd.read_csv("PATH_TO_TRANSFORMERS_CANDIDATES/leaderboard_candidates_transformer.csv")
submission_df_tr["score"]=submission_df_tr["score"].map(ast.literal_eval)
submission_df_tr["code"]=submission_df_tr["code"].map(ast.literal_eval)
submission_df_tr["rank"]=submission_df_tr["rank"].map(ast.literal_eval)
submission_df_tr = submission_df_tr.explode(['code','score', 'rank'])
submission_df_tr[["session_id","code","rank"]]=submission_df_tr[["session_id","code","rank"]].astype(int)
submission_df_tr.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_transformer","score":"score_transformer"},inplace=True)

In [31]:
dataset=dataset.merge(submission_df,on=["Session_Id","Item_ID"],how="outer")
del submission_df
dataset=dataset.merge(submission_df_tr,on=["Session_Id","Item_ID"],how="outer")
del submission_df_tr

In [32]:
dataset=dataset.merge(dataset_max,on=["Session_Id"],how="left")
del dataset_max

In [33]:
dataset.fillna(0,inplace=True)

In [35]:
# dataset_macro_features = pd.read_csv('./macro-feats/test_leaderboard_with_macro_feats.csv',
#                                      usecols=['session_id',
#                                                 'session_similarity',
#                                                 'session_similarity_uniques', 'session_similarity_feats', 'session_similarity_feats_uniques',
#                                               'length_of_session_seconds',
#                    'avg_time_spent_per_item_seconds',
#                    'variance_time_spent_per_item_seconds',
#                    'n_seen_items',
#                    'n_unique_items',])
#
# dataset_macro_features.rename(columns={'session_id':'Session_Id'}, inplace=True)
# dataset = dataset.merge(dataset_macro_features, on='Session_Id', how='left')

In [37]:
dataset.drop('Max_Score_TopPop15', axis=1, inplace=True)

In [38]:
dataset.head()

,Item_ID,Score_UCF,Score_ICBF,Score_Graph,Score_ICF,Score_TopPop15,Session_Id,score,rank,Max_Score_UCF,Max_Score_ICBF,Max_Score_Graph,Max_Score_ICF
0,21,0.100227,0.0,0.019740,0.000000,0.000000,26,0.001049,44,55.290478,3.397799,2.522003,0.330278
1,52,0.242711,0.0,0.087397,0.548803,0.000000,26,0.001091,40,55.290478,3.397799,2.522003,0.330278
2,80,0.090264,0.0,0.045723,0.545978,0.000000,26,0.000955,49,55.290478,3.397799,2.522003,0.330278
3,115,0.025982,0.0,0.000000,0.000000,0.142399,26,0.000000,101,55.290478,3.397799,2.522003,0.330278
4,159,0.058162,0.0,0.020845,0.000000,0.000000,26,0.001075,42,55.290478,3.397799,2.522003,0.330278


In [39]:
gc.collect()

0

In [40]:
dataset=dataset.sort_values("Session_Id")

In [41]:
gc.collect()

0

In [42]:
import lightgbm

In [43]:
k=10

In [44]:
for index in range(k):
    model = lightgbm.Booster(model_file=f'./{k}_fold/model_{index}.txt')
    dataset[f"score_{index}"]=model.predict(dataset.drop(["Session_Id","Item_ID"],axis=1))

#combine scores
dataset["score"]=dataset[[f"score_{i}" for i in range(k)]].apply(lambda x:np.mean(x),axis=1)
#dataset["score"]=dataset[[f"score_{i}" for i in range(k)]].apply(lambda x:np.median(x),axis=1)

In [45]:
dataset=dataset[["Session_Id","Item_ID","score"]]
gc.collect()

4

In [46]:
dataset.head()

,Session_Id,Item_ID,score
0,26,21,-1.120692
28683079,26,1136,-3.147938
28683078,26,1052,-2.392654
28683077,26,949,-2.392654
28683076,26,786,-2.728708


In [47]:
dataset=dataset.groupby(['Session_Id']).agg(list).reset_index()

In [48]:
def get_indices(x):
    x=np.array(x)
    indices_unsorted=np.argpartition(x, -100)[-100:]
    x=x[indices_unsorted]
    indices=indices_unsorted[np.argsort(-x)]
    return indices

In [49]:
dataset["indices"]=dataset["score"].apply(lambda x: get_indices(x))

In [50]:
dataset.iloc[0]

Session_Id                                                   26
Item_ID       [21, 1136, 1052, 949, 786, 772, 719, 707, 699,...
score         [-1.120691750730812, -3.1479380862717763, -2.3...
indices       [529, 485, 684, 649, 643, 665, 658, 499, 551, ...
Name: 0, dtype: object

In [51]:
dataset.iloc[0]["Item_ID"][536]

1686

In [52]:
def get_items(x):
    items=np.array(x[0])
    return items[x[1]]

In [53]:
dataset["predictions"]=dataset[["Item_ID","indices"]].apply(get_items,axis=1)

In [54]:
dataset=dataset[["Session_Id","predictions"]]
dataset["rank"]=dataset.apply(lambda x:tuple([rank for rank in range(1, 101)]) ,axis=1)

In [55]:
dataset

,Session_Id,predictions,rank
0,26,"[606, 1008, 4952, 4685, 4841, 4881, 4462, 1337...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,200,"[772, 1827, 270, 3400, 3893, 4878, 3424, 4024,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,205,"[2289, 3398, 4986, 1181, 1952, 2075, 2780, 384...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,495,"[4845, 4262, 3815, 4900, 2378, 3550, 2785, 445...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,521,"[1004, 3172, 143, 1165, 163, 3180, 1768, 25, 3...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
...,...,...,...
49995,4439446,"[2800, 4810, 3867, 4835, 2258, 2605, 2734, 949...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49996,4439458,"[1875, 3880, 3153, 2877, 3155, 2216, 867, 3046...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49997,4439550,"[2638, 625, 4779, 989, 1901, 4846, 2835, 2431,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49998,4439653,"[3531, 2266, 1015, 2596, 3665, 989, 360, 2638,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [56]:
dataset["predictions"]=dataset["predictions"].apply(tuple)

In [57]:
dataset=dataset[["Session_Id","predictions","rank"]]

In [58]:
dataset.columns=["session_id","code","rank"]

In [59]:
dataset.head()

,session_id,code,rank
0,26,"(606, 1008, 4952, 4685, 4841, 4881, 4462, 1337...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,200,"(772, 1827, 270, 3400, 3893, 4878, 3424, 4024,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,205,"(2289, 3398, 4986, 1181, 1952, 2075, 2780, 384...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,495,"(4845, 4262, 3815, 4900, 2378, 3550, 2785, 445...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,521,"(1004, 3172, 143, 1165, 163, 3180, 1768, 25, 3...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [60]:
submission_df = dataset.explode(['code', 'rank']).astype(int)

In [62]:
mapping= pd.read_csv('./dressipi_dataset/map.csv')
submission_df=submission_df.merge(mapping,on="code",how="left")
del submission_df["code"]
submission_df=submission_df[["session_id","item_id","rank"]]

In [63]:
print(len(submission_df))

5000000


In [64]:
submission_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000000 entries, 0 to 4999999
Data columns (total 3 columns):
 #   Column      Dtype
---  ------      -----
 0   session_id  int64
 1   item_id     int64
 2   rank        int64
dtypes: int64(3)
memory usage: 152.6 MB


In [65]:
submission_df.isna().sum()

session_id    0
item_id       0
rank          0
dtype: int64

In [66]:
submission_df.to_csv(
    f'predictions_leaderboard_{k}_fold.csv',
    index=False
    )